
# torchtune 学习笔记

> 按“入门 → 微调配方 → 基础组件 → 教程 → 深度解析 → API 参考”的结构梳理 torchtune。Notebook 既可当作查询手册，也可在本地执行示例命令验证环境。


## Getting Started 入门指南


### torchtune Overview · torchtune 概述
- **定位**：基于 PyTorch 的原生 LLM 微调与实验框架，强调模块化、开箱即用与高度可扩展性。
- **核心价值**：
  1. **Simplicity & Extensibility**：组件化设计，无额外训练框架依赖，便于复用与自定义。
  2. **Correctness**：组件、模型与配方经过严密的数值/性能对齐测试。
  3. **Stability**：遵循 “PyTorch 就该一用即得” 的理念，同时支持多硬件与分布式训练。
  4. **Democratizing Fine-tuning**：LoRA、QLoRA、FSDP2 等工具降低算力门槛，让个人和小团队也能玩转 LLM。
- **生态整合**：原生支持 Hugging Face Datasets、EleutherAI Eval Harness、torchao 量化、FSDP2 等。


### Install Instructions · 安装说明
- **基础安装**：
  ```bash
  pip install torchtune
  ```
- **可选依赖**：
  - `bitsandbytes`：用于 QLoRA / 4bit 权重量化。
  - `torchao`：量化感知训练 (QAT) 所需。
  - `wandb` / `comet-ml`：云端监控与实验管理。
- **环境需求**：
  - 建议使用与 PyTorch 匹配的 CUDA/cuDNN 版本。
  - 使用 QAT / torchao 功能需 `torch>=2.4.0`。
  - LoRA/QLoRA 推荐显存 ≥24GB；分布式或大模型需多卡/多机资源。

In [ ]:

# 检查 torchtune 是否已安装
try:
    import torchtune
    print('torchtune 版本:', torchtune.__version__)
    print('安装路径:', torchtune.__file__)
except ImportError as err:
    print('未检测到 torchtune ->', err)
    print('安装示例: pip install torchtune')


### Fine-Tune Your First LLM · 微调你的第一个 LLM
1. **下载预训练模型**（以 Llama2-7B 为例）：
   ```bash
   tune download meta-llama/Llama-2-7b-hf      --output-dir /tmp/Llama-2-7b-hf      --hf-token <HF_ACCESS_TOKEN>
   ```
2. **选择配方与配置**：
   - `tune ls` 列出配方 (Recipe) 与对应配置 (Config)。
   - 示例：`lora_finetune_single_device` + `llama2/7B_lora_single_device`。
3. **视需求修改配置**：
   - 命令行覆盖：`tune run ... epochs=1 learning_rate=1e-4`。
   - 或 `tune cp llama2/7B_lora_single_device my_config.yaml` 后编辑。
4. **启动微调**：
   ```bash
   tune run lora_finetune_single_device      --config llama2/7B_lora_single_device      epochs=1
   ```
5. **监控与评估**：关注 Loss、GPU 内存、日志文件；根据项目需求调整数据、超参与监控工具。


### torchtune CLI · torchtune 命令行界面
- **`tune --help`**：查看全量命令列表。
- **常用子命令**：
  | 命令 | 功能 |
  | --- | --- |
  | `tune download` | 从 Hugging Face / Kaggle 拉取模型权重与 tokenizer。 |
  | `tune ls` | 列出内置配方与配置。 |
  | `tune cp` | 复制配方或配置到本地，方便自定义。 |
  | `tune run` | 运行配方；可附 torchrun 参数、命令行覆盖项。 |
  | `tune validate` | 检查 YAML 配置格式合法性。 |
  | `tune cat` | 美观输出配置内容，支持 `--sort`。 |
- **分布式示例**：
  ```bash
  tune run --nnodes=2 --nproc-per-node=4 full_finetune_distributed     --config llama3/8B_full
  ```
- **覆盖参数**：命令末尾追加 `key=value`，如 `optimizer.lr=5e-5`、`model.lora_rank=64`。

In [ ]:

import shutil
if shutil.which('tune'):
    print('已检测到 tune CLI，可尝试 `!tune --help`')
else:
    print('未检测到 tune CLI，安装 torchtune 后会自动提供')


## Finetuning Recipes 微调配方


### Recipes Overview · 配方概述
- **概念**：配方是一份可运行的训练/评估脚本，内部连接模型、数据、优化器、日志等组件。
- **组成**：
  1. **Config**：YAML 文件，描述组件与超参。
  2. **Recipe Script**：CLI 入口，解析配置、设置环境。
  3. **Recipe Class**：核心训练流程（迭代、梯度更新、日志记录）。
- **类型**：
  - 全参数微调（single / distributed）。
  - 参数高效微调（LoRA、QLoRA、DoRA）。
  - 对齐 / 偏好微调（DPO、RRHF）。
  - 量化 / 量化感知训练（QAT、post-training quantize）。
  - 蒸馏、评估、推理等特殊流程。


### LoRA Single Device Finetuning · LoRA 单设备微调
- **场景**：单卡显存有限但希望微调大模型。
- **示例命令**：
  ```bash
  tune download meta-llama/Meta-Llama-3.1-8B-Instruct     --output-dir /tmp/Meta-Llama-3.1-8B-Instruct     --ignore-patterns "original/consolidated.00.pth"

  tune run lora_finetune_single_device     --config llama3_1/8B_lora_single_device
  ```
- **常用覆盖项**：
  ```bash
  tune run lora_finetune_single_device     --config llama3_1/8B_lora_single_device     model.lora_attn_modules="[q_proj,k_proj,v_proj]"     model.apply_lora_to_mlp=True     model.lora_rank=64     model.lora_alpha=128
  ```
- **支持的 PEFT**：LoRA、QLoRA、DoRA，可根据内存/性能需求选择。
- **教程参考**：LoRA 微调 Llama2、QLoRA 微调、Chat 数据微调。


### Distributed Quantization-Aware Training (QAT) · 分布式量化感知训练
- **目的**：在训练阶段模拟量化效果，获得高精度量化模型。
- **推荐流程**：
  ```bash
  tune download meta-llama/Meta-Llama-3-8B-Instruct     --output-dir /tmp/Meta-Llama-3-8B-Instruct     --ignore-patterns "original/consolidated.00.pth"     --hf-token <TOKEN>

  tune run --nproc-per-node 6 qat_distributed     --config llama3/8B_qat_full
  ```
- **关键参数**：
  - `fake_quant_after_n_steps` 控制延迟生效的步数，常取总步数约一半。
  - `quantizer.*` 设定量化策略（需 `torch>=2.4.0` 支持 Int8DynActInt4WeightQATQuantizer）。
- **后续步骤**：使用 `tune run quantize` 生成最终量化模型，再开展评估/部署。


### Direct Preference Optimization · 直接偏好优化
- **用途**：根据偏好数据优化模型输出，常用于模型对齐（安全、诚实、有用）。
- **配方**：`lora_dpo_single_device`、`lora_dpo_distributed`、`full_dpo_distributed`。
- **示例**：
  ```bash
  tune run lora_dpo_single_device     --config llama3_1/8B_lora_dpo_single_device     loss=torchtune.modules.loss.RSOLoss     gamma=0.5
  ```
- **损失函数**：
  - DPO Loss（默认）。
  - RSO / Hinge Loss 等，可通过 `loss=` 切换。
- **数据准备**：需将数据整理为 `chosen` 与 `rejected` 对，并可搭配 SFT 结果作为初始化。


## Basics 基础


### Datasets Overview · 数据集概览
- 封装常见任务数据，并支持自定义数据接入。
- 与 `torchtune.data` 配合使用，实现统一的数据加载、转换、打包流程。
- 可直接引用 Hugging Face Datasets，或构建本地数据集类。


### Chat Datasets · 聊天数据集
- 结构多为 `system` / `user` / `assistant` 消息列表。
- 通常搭配 `Message` 对象与 prompt 模板使用。
- 应用：Chat SFT、对话式模型对齐、Agent 微调。


### Instruct Datasets · 指令数据集
- 以 `instruction` + `response` 或 `input` + `output` 形式组织。
- 常用于指令跟随模型训练、任务式问答。
- 可与 Chat 数据一样转化为消息格式或自定义 collator。


### Multimodal Datasets · 多模态数据集
- 支持图文、语音等多模态输入。
- 搭配 `MultimodalTransform`（例如 CLIP 图像编码 + 文本拼接）统一处理。
- 需留意设备与 dtype 兼容性，以及多模态数据的批量对齐。


### Preference Datasets · 偏好数据集
- 结构包含正/负示例或排名列表，用于 DPO、RRHF、RLHF。
- 常见字段：`chosen` / `rejected`、偏好分数、元数据。
- 训练前可结合 SFT 结果提升稳定性。


### Text-completion Datasets · 文本补全数据集
- 面向传统语言模型任务（next-token prediction）。
- 数据多为长文本片段，可结合 sample packing 提升吞吐量。
- Collate 时需处理 EOS、padding、attention mask 等。


### Multimodal Transforms · 多模态转换
- 提供图像/音频编码、模态融合、prompt 拼接等操作。
- 可按需定制（例如图像编码后接入文本 prompt）。


### Messages · 消息
- 采用 `Message(role, content, name=None, metadata=None)` 结构。
- 统一 Chat 数据的表示，便于后续处理。


### Message Transforms · 消息转换
- 对 `Message` 列表执行插入系统提示、截断、拼接等操作。
- 可组合成流水线，例如追加安全提示、格式化用户输入。


### Tokenizers · 分词器
- 支持 Hugging Face Tokenizer、SentencePiece 等。
- 可设置 `max_seq_len`、特殊 token、padding 方式。
- 与 prompt 模板、sample packing 协同工作。


### Prompt Templates · 提示模板
- 统一组织 system / user / assistant 文本，避免提示词格式混乱。
- 可定义多种模板（Chat、Instruct、多模态），在配置中引用。


### Sample Packing · 样本打包
- 将多个短样本拼接成长序列，减少 padding、提高 GPU 利用率。
- 需谨慎处理样本边界，保证 label mask 正确。


### Custom Components and Recipes · 自定义组件与配方
- 可继承现有模块（模型、loss、scheduler）或编写新组件。
- 复制内置 Recipe 脚本进行修改（`tune cp`）。
- 适合引入自定义评估、复杂数据流程或第三方工具。


## Tutorials 教程


### Meta Llama3 in torchtune · torchtune 中的 Meta Llama3
- 讲解 Llama3 模型下载、配置选择、环境准备。
- 提示如何申请 Hugging Face 访问权限与 token。
- 演示 `tune download`、`tune ls`、`tune run` 的完整流程。


### Fine-Tuning Llama3 with Chat Data · 使用 Chat 数据微调 Llama3
- 展示如何组织 Chat 数据、转换为 Message 序列。
- 说明 prompt 模板、最大长度、batch size 等配置要点。
- 演示 LoRA/全量微调在聊天场景的差异。


### Fine-Tuning Llama2 with LoRA · 使用 LoRA 微调 Llama2
- 通过 LoRA/DoRA 减少可训练参数，适合单卡微调。
- 涉及 rank、alpha、dropout 等超参调优策略。
- 给出日志监控与性能对齐建议。


### Fine-Tuning Llama2 with QLoRA · 使用 QLoRA 微调 Llama2
- 结合 4bit 权重量化的 LoRA，显著降低显存需求。
- 介绍 `bitsandbytes` 依赖、量化精度选择、梯度累积搭配。


### Fine-Tuning Llama3 with QAT · 使用 QAT 微调 Llama3
- 演示如何使用 torchao 实现量化感知训练。
- 重点介绍 fake quant 延迟参数、训练→量化→评估的完整链路。


### End-to-End Workflow with torchtune · 使用 torchtune 的端到端工作流程
- 涵盖数据准备、模型下载、训练、评估、日志、部署的完整案例。
- 强调如何妥善管理配置、版本与实验记录。


### Distilling Llama3.1 8B into Llama3.2 1B using Knowledge Distillation · 使用知识蒸馏将 Llama3.1 8B 蒸馏为 Llama3.2 1B
- 介绍 Teacher/Student 配置、蒸馏损失、温度等关键超参。
- 演示如何复用已有评估脚本验证 Student 效果。


### Memory Optimization Overview · 内存优化概述
- 系统列出内存优化手段：LoRA/QLoRA、激活检查点、梯度累积、混合精度、FSDP2 等。
- 提供选择策略与组合示例。


### Multi-node finetuning · 多节点微调
- 指导多节点/多卡训练的参数配置、网络环境要求、日志/检查点评估策略。
- 给出常见故障排查建议。


## Deep-Dives 深度解析


### Checkpointing in torchtune · 保存检查点
- 讲解单机/分布式场景下的 checkpoint 策略、恢复流程与注意事项。
- 说明如何与数据加载器的状态恢复协同。


### All About Configs · 关于配置
- 深入解析 YAML 配置结构、组件注册、参数覆盖、继承关系。
- 提供命名规范与版本控制建议。


### What Are Recipes? · 什么是配方？
- 阐述配方的分层结构：CLI 入口、Recipe 类、组件组合。
- 对比单机/分布式配方的差异并解释保持简单的原因。


### Logging to Comet · 记录到 Comet
- 展示如何在配置中启用 Comet 记录器、同步指标与超参。


### Logging to Weights & Biases · 记录到 Weights & Biases
- 介绍 WandB 集成方式，包括 API key 管理、日志字段与可视化面板。


## API Reference API 参考


| 模块 | 功能简述 |
| --- | --- |
| `torchtune.config` | 配置系统与组件注册表，负责 YAML 解析与依赖注入。 |
| `torchtune.data` | 数据流水线基础构件，含 Message、Transforms、Collator、DataModule。 |
| `torchtune.datasets` | 内置与第三方数据集适配层，支持多种任务格式。 |
| `torchtune.generation` | 推理/生成工具，包含采样策略、解码器封装、logits 处理。 |
| `torchtune.models` | 主流 LLM 实现（Llama、Mistral、Phi、Mixtral 等）及权重转换工具。 |
| `torchtune.modules` | 可复用模块（Attention、MLP、Position Embedding、LoRA/QLoRA 层等）。 |
| `torchtune.rlhf` | 偏好对齐模块（Reward Model、DPO/RSO/RLHF Loss、采样策略）。 |
| `torchtune.training` | 训练循环、优化器、调度器、量化器、FSDP2 等。 |
| `torchtune.utils` | 工具集合：分布式、日志、检查点、随机种子、profiling、metrics 等。 |

> 最新 API 细节请参考官方文档或直接阅读源码。